<a href="https://colab.research.google.com/github/kvinne-anc/Tanzania-Soil-Topography-Waterpumps-Project/blob/main/TanzaniaTopSoilData_LogRegModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

In [ ]:
%%time 

# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
python3-gdal is already the newest version (2.2.3+dfsg-2).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-rtree is already the newest version (0.8.3+ds-1).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.
  Cloning git://github.com/geopandas/geopandas.git to /tmp/pip-req-build-s1f5lqcm
  Running command git clone -q git://github.com/geopandas/geopandas.git /tmp/pip-req-build-s1f5lqcm
  Created wheel for geopandas: filename=geopandas-0.8.0+42.g16f3ba8-py2.py3-none-any.whl size=963769 sha256=546cab6d440ade31dd137c41d2eaec5d2e0860fa87b0841f055f2db6d3a51a0a
  Stored in directory: /tmp/pip-ephem-wheel-cache-3etpznoc/wheels/91/24/71/376c9c67192694168352

In [ ]:
import pandas as pd
import geopandas as gdp 
from shapely.geometry import Point
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
from sklearn.model_selection import train_test_split


train = pd.merge(pd.read_csv('/content/tza_waterpumps.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'), on= 'id')
test = pd.read_csv('/content/test_tza_waterpumps.csv')

sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')


ParserError: ignored

In [ ]:
# Split train into train & val
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['status_group'], random_state=42)

In [ ]:

train.shape, val.shape, test.shape

((12383, 47), (3096, 47), (14118, 46))

In [ ]:
train['status_group'].value_counts(normalize=True)

functional                 0.524452
non functional             0.411461
functional needs repair    0.064087
Name: status_group, dtype: float64

In [ ]:
cat_col = train.columns[train.dtypes=='object']

(cat_col)

Index(['date_recorded', 'funder', 'installer', 'wpt_name', 'basin',
       'subvillage', 'region', 'lga', 'ward', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group', 'SOIL_TYPE',
       'MAIN_TYPE', 'geometry', 'status_group'],
      dtype='object')

In [ ]:
train[cat_col] = train[cat_col].astype('category')

In [ ]:
train.loc[0]
train['lga'].cat.codes

2672      88
49889    110
52171     96
45347    121
53494     57
        ... 
52516    123
3278      11
31958     16
9083     103
41732     12
Length: 45989, dtype: int8

In [ ]:
import numpy as np

In [ ]:
trained = train.drop(['geometry', 'installer', 'extraction_type', 'construction_year', 'quantity', 'wpt_name', 'num_private', 'basin', 'region', 'district_code', 'population', 'public_meeting', 'recorded_by', 'id', 'scheme_management', 'scheme_name', 'permit', 'extraction_type_group', 'extraction_type_class', 'management_group', 'management', 'payment', 'payment_type', 'quality_group', 'source_type', 'source_class', 'source', 'date_recorded', 'waterpoint_type', 'waterpoint_type_group', 'status_group'], axis=1)

In [ ]:
trained.head()

,index_left,amount_tsh,funder,gps_height,longitude,latitude,subvillage,region_code,lga,ward,water_quality,quantity_group,AREA,PERIMETER,SOIL_TYPE,MAIN_TYPE
2672,54861.0,0.0,Oxfam,0.0,30.691264,-2.847368,Nyakuguma,18.0,Ngara,Bugarama,soft,enough,0.692150,6.306691,FRh16,Haplic Ferrasols
49889,39233.0,8000.0,Mission,1466.0,35.297519,-9.760209,Ganga,10.0,Songea Rural,Wino,soft,enough,1.112332,11.209680,AChX4,Haplic Acrisols
52171,10858.0,0.0,Watu Wa Ujerumani,1188.0,34.031656,-1.146791,Kabwana,20.0,Rorya,Mkoma,soft,enough,0.085020,1.658887,CMx3,Chromic Cambisol
45347,39840.0,100.0,Dhv,379.0,36.861544,-9.022433,Mwanza,5.0,Ulanga,Mbuga,unknown,enough,2.569950,29.917410,CMo21,Ferralic Cambiso
53494,6268.0,0.0,Netherland,0.0,33.754734,-3.157654,Nyanhumbili,17.0,Maswa,Buchambi,soft,enough,0.070808,1.707260,PLe7,Eutric Planosols


In [ ]:
numeric_features = trained.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = trained.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 10000
categorical_features = cardinality[cardinality <= 1000000].index.tolist()

# Combine the lists 

features = numeric_features + categorical_features
print(features)

['index_left', 'amount_tsh', 'gps_height', 'longitude', 'latitude', 'region_code', 'AREA', 'PERIMETER', 'funder', 'subvillage', 'lga', 'ward', 'water_quality', 'quantity_group', 'SOIL_TYPE', 'MAIN_TYPE']


In [ ]:
target = train['status_group']

In [ ]:
X_train = trained[features]
y_train = train['status_group']
X_val = val[features]
y_val = val['status_group']
X_test = test[features]

In [ ]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True), 
    SimpleImputer(strategy='mean'), 
    StandardScaler(), 
    LogisticRegression(multi_class='auto', solver='lbfgs', n_jobs=-1)
)

# Fit on train
pipeline.fit(X_train, y_train)

# Score on val
print('Validation Accuracy', pipeline.score(X_val, y_val))

# Predict on test
y_pred = pipeline.predict(X_test)